# **0. 패키지 설치**

In [ ]:
!pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.9 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# **0. 구글 드라이브 마운트 & api key**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import glob
import os

Mounted at /content/drive


In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI

#  API 키 설정
OPENAI_API_KEY = ""  # 여기에만 입력하면 아래에서 자동 사용됨
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# **1. 문서 업로드 & Retriever**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 1. 📥 PDF 로딩 (하위 폴더 포함) + 출처 추적
def load_all_pdfs(root_dir):
    all_docs = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".pdf"):
                full_path = os.path.join(dirpath, filename)
                loader = UnstructuredPDFLoader(full_path)
                docs = loader.load()

                for doc in docs:
                    # 📎 source: 상대 경로 기준으로 저장 (예: RAG/작품A.pdf)
                    relative_path = os.path.relpath(full_path, root_dir)
                    doc.metadata["source"] = relative_path
                    doc.page_content = doc.page_content.strip().replace("\n", " ").replace("  ", " ")

                all_docs.extend(docs)
    return all_docs

# 2. 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

# 3. 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. 배치 임베딩
def embed_in_batches(docs, batch_size=100):
    vectorstore = None
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        print(f"🔄 Embedding batch {i} ~ {i + len(batch)} / {len(docs)}")
        if vectorstore is None:
            vectorstore = FAISS.from_documents(batch, embedding=embeddings)
        else:
            vectorstore.add_documents(batch)
    return vectorstore

# 5. 저장 경로
VECTORSTORE_PATH = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/faiss_index_답변해설"

# 6. 벡터스토어 생성 또는 로딩
def build_vectorstore():
    root_dir = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/data/RAG"
    all_docs = load_all_pdfs(root_dir)
    split_docs = text_splitter.split_documents(all_docs)
    vectorstore = embed_in_batches(split_docs)
    vectorstore.save_local(VECTORSTORE_PATH)
    return vectorstore

# 7. 리트리버 생성
def get_retriever():
    if os.path.exists(VECTORSTORE_PATH):
        print("📁 기존 벡터스토어 불러오는 중...")
        vectorstore = FAISS.load_local(VECTORSTORE_PATH, embeddings, allow_dangerous_deserialization=True)
    else:
        print("🧠 새 벡터스토어 생성 중...")
        vectorstore = build_vectorstore()

    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.5
        }
    )
    return retriever

# 🔄 최종 retriever
retriever = get_retriever()

/tmp/ipython-input-5-2237303118.py:37: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


📁 기존 벡터스토어 불러오는 중...


# **2. 개념 답변 QA chain**

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 개념 설명 체인
concept_prompt = ChatPromptTemplate.from_template("""
너는 수능 문학 개념을 설명해주는 튜터야. 아래 질문에 대해 간단명료하게 설명해줘.

질문:
{question}
""")

concept_chain = concept_prompt | llm | StrOutputParser()

/tmp/ipython-input-6-3416333025.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)


# **3. 문제 답변&해설 QA 체인**

In [ ]:
# 문제 기반 QA RAG 체인
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda

# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# 문제 기반 QA 체인용 프롬프트
qa_prompt = ChatPromptTemplate.from_template("""
당신은 한국 수능 국어 문학 전문 튜터입니다.

다음은 수능 국어 문학 객관식 문제입니다. ①, ②, ③, ④, ⑤ 중 하나를 고르는 객관식 문제이며,
정답은 반드시 지문 및 <보기>의 정확한 분석에 근거해 판단해야 합니다.

🟨 반드시 지켜야 할 분석 기준:

1. 지문 분석을 중심으로 판단하며, **선택지의 내용이 지문과 일치하는지** 엄밀히 검토하세요.
2. 문제에 <보기>가 있다면, <보기> 속 설명(구조, 시점, 표현, 인물 해석 등)을 **지문에 어떻게 적용했는지** 구체적으로 분석하세요.
3. **서술 방식, 인물 심리 표현, 문체, 시점 변화, 지시 표현, 병렬 구조** 등은 서술상·표현상 특징 문제에서 핵심 근거입니다.
4. 참고 자료는 반드시 보조적 용도로만 활용합니다. 절대 지문을 덮어쓰거나 대체해서는 안 됩니다.

📌 답변 형식:

[정답]
- (①, ②, ③, ④, ⑤ 중 하나)

[해설]
- 문제에서 요구한 핵심 요소(예: 표현 방식, 구조, 시점 등)에 따라 왜 정답인지 논리적으로 설명합니다.
- 지문 및 <보기>의 문장을 **직접 인용**하여 명확한 판단 근거를 제시하세요.
- 나머지 오답 선택지들은 각각 왜 틀렸는지를 간단히 설명하세요.

📖 [지문]
{context}

📚 [참고 자료] — 필요 시만 사용 (retriever 제공):
{reference}

🙋‍♂️ [문제 및 <보기>]
{question}
""")

def format_with_retrieved_docs(inputs):
    question = inputs["question"]
    context = inputs["context"]

    # 🔍 관련 문서 검색 (context와 question 모두 기준으로 검색)
    retrieved_docs = retriever.get_relevant_documents(f"{context}\n\n{question}")
    retrieved_context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    return {
        "context": context,           # 지문 및 <보기>
        "reference": retrieved_context,  # 보조 자료
        "question": question
    }

# RAG QA 체인: 문항 지문 중심 + retriever 참고
rag_qa_chain = (
    RunnableLambda(format_with_retrieved_docs)
    | qa_prompt
    | llm
    | StrOutputParser()
)

# **4. 문제 vs 개념 분류 함수**

In [ ]:
def is_problem_question(question: str) -> bool:
    classification_prompt = ChatPromptTemplate.from_template("""
다음 질문이 문학 개념 질문인지, 지문 기반 문제인지 판별해줘. '개념' 또는 '문제' 중 하나로만 답해.

질문:
{question}
""")
    chain = classification_prompt | llm | StrOutputParser()
    result = chain.invoke({"question": question})
    return "문제" in result.strip()

# **5. 최종 질문 처리 함수**

In [ ]:
def tutor_response(question: str, passage: str = None):
    """
    문학 질문 또는 문항에 대한 응답을 처리합니다.

    Parameters:
    - question: 수험생 질문 또는 문학 개념 질문
    - passage: 선택적 입력. 문항 지문이 있는 경우 함께 제공
    """
    if is_problem_question(question):
        if not passage:
            print("❗ 오류: 문학 문제 풀이에는 지문(passage)이 필요합니다.")
            return

        print("📘 [문제에 대한 정답 및 해설]")

        # rag_qa_chain에 지문과 문제를 분리해서 입력
        response = rag_qa_chain.invoke({
            "context": passage,
            "question": question
        })
        print(response)

    else:
        print("📘 [문학 개념 설명]")
        response = concept_chain.invoke({"question": question})
        print(response)

# **<직접 수정한 깨끗한 텍스트로 테스트>**
- 2025학년도(2024년 시행) 6월, 9월 수능 테스트

In [ ]:
import re

# 🔸 문제/지문 파싱 함수
def extract_passage_question_pairs(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 🔹 블록 단위로 split
    raw_blocks = re.split(r"---\s+(\d{4}-(?:\d{2}|수능)-국어_[^\s(]+)", content)
    blocks = list(zip(raw_blocks[1::2], raw_blocks[2::2]))  # [(header, content), ...]

    passage = None
    pairs = []

    for header, body in blocks:
        body = body.strip()

        # 지문 블록이면 지문 갱신
        if "_p" in header:
            passage = body

        # 문제 블록이면 현재 지문과 연결
        elif re.match(r"\d{4}-(\d{2}|수능)-국어_\d{2}", header):
            question = body
            if passage:  # 지문이 있어야 유효
                pairs.append({
                    "id": header,  # e.g., "2025-06-국어_18"
                    "context": passage,
                    "question": question
                })

    return pairs

# 🔹 정답표 딕셔너리 (연도-월별 구성)
answer_key = {
    "2024-06": {
        18: "4", 19: "3", 20: "3", 21: "4", 22: "1", 23: "2",
        24: "5", 25: "2", 26: "5", 27: "2", 28: "3", 29: "4",
        30: "3", 31: "5", 32: "4", 33: "3", 34: "3"
    },
    "2024-09": {
        18: "1", 19: "3", 20: "5", 21: "5", 22: "3", 23: "1",
        24: "4", 25: "4", 26: "5", 27: "3", 28: "4", 29: "3",
        30: "5", 31: "2", 32: "1", 33: "3", 34: "3"
    },
    "2024-수능": {
        18: "2", 19: "4", 20: "1", 21: "4", 22: "4", 23: "5",
        24: "2", 25: "2", 26: "1", 27: "1", 28: "4", 29: "3",
        30: "5", 31: "4", 32: "3", 33: "5", 34: "2"
    }
}

# 🔸 회차 및 문항 번호 파싱 함수
def get_correct_answer_from_id(qid):
    # e.g., qid = "2025-06-국어_18"
    match = re.match(r"(\d{4}-\d{2}|2024-수능)-국어_(\d+)", qid)
    if not match:
        return "❓"
    key, num = match.groups()
    num = int(num)
    return answer_key.get(key, {}).get(num, "❓")

# 🔹 문항별 tutor_response 실행
def run_tutor_response_on_question_pairs(pairs):
    for pair in pairs:
        qid = pair["id"]
        correct_answer = get_correct_answer_from_id(qid)
        print(f"\n\n=============================== ✅ 문항 {qid} (정답: {correct_answer}) ===============================\n")
        try:
            tutor_response(
                question=pair["question"],
                passage=pair["context"]
            )
        except Exception as e:
            print(f"❌ 문항 {qid} 처리 중 오류 발생: {e}")

In [ ]:
# 🔸 txt 파일 경로
txt_path = "/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/문학/data/문제해설모델 테스트_직접수정.txt"

# 🔹 전체 문항 불러오기 (지문과 매핑된)
qa_inputs = extract_passage_question_pairs(txt_path)

# ✅ 실행
run_tutor_response_on_question_pairs(qa_inputs)



=============================== ✅ 문항 2024-06-국어_18 (정답: 4) ===============================

📘 [문제에 대한 정답 및 해설]


/tmp/ipython-input-7-3533957196.py:50: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(f"{context}\n\n{question}")


[정답]
- ④

[해설]
- ④번 선택지는 "㉣은 이대봉이 중원으로 향하기 전에 머물던 공간이다."라고 설명하고 있습니다. 지문에서 "서해 용왕의 구함을 입어 살아나 서역 천축국 ㉣(백운암)에 가 팔 년을 의탁하였나이다."라는 부분이 있습니다. 이 부분에서 이대봉이 서역 천축국 백운암에 머물렀다는 사실을 알 수 있으며, 이후 중원으로 향하기 전에 머물렀던 공간임을 알 수 있습니다. 따라서 ④번 선택지가 가장 적절합니다.

- ①번 선택지는 "㉠은 이대봉이 이릉의 영혼을 만나 갑옷과 칼을 얻은 공간이다."라고 설명하고 있습니다. 그러나 지문에서 ㉠은 용궁으로, 이대봉이 부친과 함께 떠난 장소로 설명되어 있으며, 이릉의 영혼을 만난 장소는 농서입니다. 따라서 ①번은 틀렸습니다.

- ②번 선택지는 "㉡은 흉노가 침범한 곳이자 이대봉이 흉노를 처단한 공간이다."라고 설명하고 있습니다. 그러나 지문에서 ㉡은 황성으로, 이대봉이 부친과 함께 머물 곳을 정한 후 성상께 상소를 올린 장소로 설명되어 있습니다. 따라서 ②번은 틀렸습니다.

- ③번 선택지는 "㉢은 장 한림 부부가 간신의 모해로 유배 간 공간이다."라고 설명하고 있습니다. 그러나 지문에서 ㉢은 해도로, 이대봉 부자가 유배 간 곳으로 설명되어 있습니다. 따라서 ③번은 틀렸습니다.

- ⑤번 선택지는 "㉤은 동돌수가 이대봉을 피해 달아난 공간이다."라고 설명하고 있습니다. 그러나 지문에서 ㉤은 금릉으로, 황성이 함몰된 후 어가가 이동한 장소로 설명되어 있습니다. 따라서 ⑤번은 틀렸습니다.


=============================== ✅ 문항 2024-06-국어_19 (정답: 3) ===============================

📘 [문제에 대한 정답 및 해설]
[정답]
- ③

[해설]
- ③ 선택지는 부친이 세상을 버린 이유를 "혼약이 어그러진 것과 이 시랑의 죽음에 대한 분노 때문"이라고 설명하고 있습니다. 그러나 지문에서는 부친이 세상을 버린 이유를 "대인의 억울함과 소첩의 앞길

# **<테스트 결과>** 최소 8문제~최대 10문제

❌ 오답 문항 수

- 2025년 6월: 3개 (20, 26, 34)

- 2025년 9월: 2개 (18, 27)

- 2025년 수능: 4개 (20, 21, 26, 32)

- 총 오답: 9문항

🧾 요약
- 총 문항 수: 51

- 정답 수: 42

- 오답 수: 9

### **정답률: 약 82.4%**
- 오답 중 정답률이 50% 아래인 문제들이 많으며, 모델이 두번째로 높은 선택비율의 정답을 선택한 것이 많음. => LLM의 추론 능력 부족
- 오답 중 대부분이 <보기>가 포함된 문제 => 보기의 조건을 지문에 정확히 적용해야 하는 문제에서 약한 경향이 있어보임